In [1]:
import os
import neurokit2 as nk
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df1 = pd.read_csv('1-FirstTenSignals.csv')
df2 = pd.read_csv('2-SecondTenSignals.csv')
df3 = pd.read_csv('3-ThirdTenSignals.csv')
df4 = pd.read_csv('4-FourthTenSignals.csv')
df5 = pd.read_csv('5-FifthTenSignals.csv')
df6 = pd.read_csv('6-SixthTenSignals.csv')
df7 = pd.read_csv('7-SeventhTenSignals.csv')
df8 = pd.read_csv('8-LastTenSignals.csv')

In [3]:
df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=1)
df.head()

,I01,I02,I03,I04,I05,I06,I07,I08,I09,I10,...,I66,I67,I68,I69,I70,I71,I72,I73,I74,I75
0,3.970588,2.333333,3.966378,3.379443,-1.891451,1.758333,-0.195833,3.284109,3.925197,1.775591,...,-2.256822,1.783307,1.428295,-1.197674,1.641129,-0.790323,-2.427093,-1.888993,2.593137,-1.468954
1,3.954248,2.330065,3.978866,3.378482,-1.892411,1.770833,-0.187500,3.268058,3.937008,1.771654,...,-2.317817,1.807384,1.439922,-1.191860,1.608871,-0.790323,-2.428975,-1.890875,2.583333,-1.470588
2,3.934641,2.297386,3.970221,3.381364,-1.890490,1.775000,-0.204167,3.264848,3.956693,1.771654,...,-2.341894,1.852327,1.453488,-1.187984,1.600806,-0.790323,-2.424271,-1.889934,2.580065,-1.472222
3,3.950980,2.281046,3.961575,3.381364,-1.887608,1.766667,-0.200000,3.264848,3.984252,1.783465,...,-2.346709,1.868379,1.445736,-1.186047,1.596774,-0.798387,-2.423330,-1.891816,2.588235,-1.473856
4,4.013072,2.290850,3.932757,3.365034,-1.883766,1.787500,-0.191667,3.258427,4.000000,1.755906,...,-2.306581,1.842697,1.443798,-1.178295,1.584677,-0.802419,-2.424271,-1.897460,2.596405,-1.475490


In [34]:
nomi_col = df.columns[73:]

In [32]:
freq = 257

## Funzione per estrarre i picchi e divirede i segnali in heartbeats salvandoli in due dataset

In [26]:
def divide_signals(signal, name, freq = 257):
    
    exps_dir = os.path.join('HB_' + name) #modify the test folder
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)
    
    _, rpeaks = nk.ecg_peaks(signal, sampling_rate=freq)
    picchi = rpeaks['ECG_R_Peaks']
    
    length = len(picchi)

    sum = 0

    for i in range(length - 1):
        sum += picchi[i + 1] - picchi[i]

    semi_period = int(sum/(length - 1)/2)
    period = semi_period*2+1
    
    print("numero battiti: " + str(length) + "\n" + "periodo: " + str(period))
    
    single_hb = np.zeros([period,length])
    # ogni colonna è un battito

    for i in range(length - 1):
        single_hb[:,i] = signal[picchi[i]-semi_period:picchi[i]+semi_period + 1]
        
    for i in range(length):
        vertical_shift = (single_hb[0,i] + single_hb[-1,i])/2
        single_hb[:,i] -= vertical_shift
        
    # creating dataset with divided signals    
    hb_dic = {}

    for i in range(length -1):
        hb_dic[i] = single_hb[:,i]

    HB_df = pd.DataFrame(hb_dic)
    
    HB_df.to_csv(exps_dir + '/Signals.csv', index = False)
       
    # estraggo tutti gli altri punti di interesse
    _, waves_dwt = nk.ecg_delineate(signal, rpeaks, sampling_rate=freq, method="dwt", show=False, show_type='peaks')
    
    wave_peak_wrt_r = {}

    chiavi = list(waves_dwt.keys())

    for i in range(10):
        wave_peak_wrt_r[chiavi[i]] = waves_dwt[chiavi[i]] - picchi
        
    df_peaks = pd.DataFrame(wave_peak_wrt_r)
    
    df_peaks.to_csv(exps_dir + '/Peaks.csv', index = False)

In [35]:
for col in nomi_col:
    divide_signals(signal = df[col], name = col)

numero battiti: 77
periodo: 199
numero battiti: 71
periodo: 217


In [ ]:
# casi non funzionanti il 16, 17, 18, 20, 22, 25, 30, 43, 44, 46, 51, 61, 73, 
